In [1]:
!pip install pandas psycopg2-binary python-dotenv

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 1.1 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 1.1 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 1.1 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.0 MB/s eta 0:00:02
   ----------------------- ---------------- 1.6/2.7 MB 1.1 MB/s eta 0:00:02
   --------------------------- ------------ 1.8/2.7 MB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 1.2 MB/s eta 0:00:00


In [2]:
# ##  Step 1. Setup and Import Libraries
import os
import pandas as pd
import psycopg2
from psycopg2 import extras
from dotenv import load_dotenv
from io import StringIO
from pathlib import Path

In [3]:
# ##  Step 2. Load Environment Variables

# Load variables from .env file
load_dotenv()

# Database settings
DB_NAME = os.getenv("PG_DBNAME")
DB_USER = os.getenv("PG_USER")
DB_PASSWORD = os.getenv("PG_PASSWORD")
DB_HOST = os.getenv("PG_HOST")
DB_PORT = os.getenv("PG_PORT")

In [4]:
# File settings
CSV_PATH = os.getenv("CSV_PATH")
CHUNK_SIZE = int(os.getenv("CHUNK_SIZE", "200000"))
TABLE_NAME = os.getenv("TABLE_NAME", "people")

print(" Environment variables loaded successfully!")


 Environment variables loaded successfully!
